In [2]:
###csv file load

In [6]:
!pip install pandas

In [3]:
import pandas as pd

In [7]:
dataset = pd.read_csv("C:/Users/anafa/OneDrive/Desktop/mainprojectdataset/bbc-news-data.csv", delimiter='\t')
dataset.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [15]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Download NLTK data if not already done
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize the lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Step 1: Load the dataset
file_path = 'C:/Users/anafa/OneDrive/Desktop/mainprojectdataset/bbc-news-data.csv'
articles = pd.read_csv(file_path, delimiter='\t')

# Step 2: Handle missing values
articles['content'] = articles['content'].fillna('')
articles['category'] = articles['category'].fillna('Unknown')

# Step 3: Clean the text data
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A) 
    text = text.lower()
    text = text.strip()
    return text

# Apply text cleaning to the 'content' column
articles['cleaned_content'] = articles['content'].apply(clean_text)

# Step 4: Tokenization and Lemmatization
def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(lemmatized_tokens)

# Apply tokenization and lemmatization
articles['processed_content'] = articles['cleaned_content'].apply(tokenize_and_lemmatize)

# Step 5: Combine category and processed content for further analysis
articles['combined_features'] = articles['category'] + ' ' + articles['processed_content']

# Save the preprocessed data to a new CSV file
preprocessed_file_path = 'preprocessed_bbc_news_data.csv'
articles.to_csv(preprocessed_file_path, index=False)

# Display the first few rows of the processed data
print(articles[['category', 'combined_features']].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anafa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anafa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anafa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   category                                  combined_features
0  business  business quarterly profit u medium giant timew...
1  business  business dollar hit highest level euro almost ...
2  business  business owner embattled russian oil giant yuk...
3  business  business british airway blamed high fuel price...
4  business  business share uk drink food firm allied domec...


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load the preprocessed dataset
file_path = 'C:/Users/anafa/OneDrive/Desktop/mainprojectdataset/preprocessed_bbc_news_data.csv'  # Path to the preprocessed dataset
articles = pd.read_csv(file_path)

# Step 2: Use TF-IDF to vectorize the 'combined_features' column
tfidf = TfidfVectorizer(stop_words='english')

# Convert the combined features to a TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(articles['combined_features'])

# Step 3: Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Step 4: Function to get recommendations based on content
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the article that matches the title
    idx = articles[articles['title'] == title].index[0]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the articles based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the 5 most similar articles (excluding the input article)
    sim_scores = sim_scores[1:6]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]

    # Return the titles and categories of the most similar articles
    return articles[['title', 'category']].iloc[article_indices]

# Example usage:
article_title = "Call centre users 'lose patience'"  # Replace with the title of an article from your dataset
recommendations = get_recommendations(article_title)
print(f"Recommendations for '{article_title}':\n", recommendations)


Recommendations for 'Call centre users 'lose patience'':
                                    title  category
316    Christmas shoppers flock to tills  business
508      Euro firms miss out on optimism  business
2011    Call for action on internet scam      tech
2057  Mobile multimedia slow to catch on      tech
2212  Mobile multimedia slow to catch on      tech
